In [2]:
import requests
import pandas as pd
from pandas import DataFrame
import json
import pickle
import datetime as dt

### SolarGeneration Forecast - Malpas-Trenton, Australia

In [3]:
lat = ("142.15")
lon = ("19.92")

url = ("http://www.7timer.info/bin/meteo.php?lon=" +
           str(lon) + "&lat=" + str(lat)
           +"&ac=0&unit=British&output=json&tzshift=0")
    
data = requests.get(url).json()
data_size = len(data["dataseries"])

# res = [[key for key in data["dataseries"][0].keys()], *[list(idx.values()) for idx in data["dataseries"] ]]
    
# clouds = res[2][0]
    

temp = []
cloud = []
rain =[]
    
for item in data["dataseries"]:
    temp.append(item["temp2m"])
    cloud.append(item["cloudcover"])
    rain.append(item["prec_amount"])

temp = pd.DataFrame(temp, columns = ['temp_avg']) 
cloud = pd.DataFrame(cloud, columns = ['Cloud Cover Percentage']) 
rain = pd.DataFrame(rain, columns = ['Rainfall in mm']) 


In [4]:
solargeneration = pd.concat([temp, cloud, rain], axis=1)
solargeneration


,temp_avg,Cloud Cover Percentage,Rainfall in mm
0,21,1,0
1,23,1,0
2,30,1,0
3,33,2,0
4,29,5,1
...,...,...,...
59,30,9,6
60,27,9,6
61,23,7,6
62,22,3,6


In [5]:
fileObj = open('modelsolar.obj', 'rb')
modelsolar = pickle.load(fileObj)
fileObj.close()
print(modelsolar)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=31, n_jobs=-1,
                      oob_score=False, random_state=None, verbose=0,
                      warm_start=False)


In [6]:
Xsolar = solargeneration[['temp_avg', 'Cloud Cover Percentage', 'Rainfall in mm']].values

In [7]:
solar_predicted_power = modelsolar.predict(Xsolar)

In [8]:
solar_predicted_power

array([1.5383871 , 1.5383871 , 1.5383871 , 2.97096774, 5.71516129,
       7.02064516, 6.28401306, 8.10032258, 9.36757143, 9.36757143,
       9.36757143, 8.10032258, 9.36757143, 8.10032258, 9.36757143,
       9.36757143, 9.36757143, 9.36757143, 9.36757143, 9.36757143,
       9.36757143, 9.36757143, 9.36757143, 9.36757143, 9.36757143,
       9.36757143, 9.36757143, 7.02064516, 7.02064516, 8.10032258,
       8.10032258, 8.10032258, 5.71516129, 7.02064516, 9.36757143,
       7.02064516, 2.95483871, 4.24225806, 9.36757143, 9.36757143,
       9.36757143, 9.36757143, 8.10032258, 9.36757143, 9.36757143,
       9.36757143, 9.36757143, 9.36757143, 9.36757143, 9.36757143,
       9.36757143, 9.36757143, 9.36757143, 9.36757143, 9.36757143,
       9.36757143, 9.36757143, 9.36757143, 9.36757143, 9.36757143,
       9.36757143, 7.02064516, 3.51741935, 5.71516129])

In [9]:
slist = solar_predicted_power.tolist()

In [10]:
dfslist = pd.DataFrame(slist, columns = ['Predicted Power'])

In [11]:
dfslist

,Predicted Power
0,1.538387
1,1.538387
2,1.538387
3,2.970968
4,5.715161
...,...
59,9.367571
60,9.367571
61,7.020645
62,3.517419


In [12]:
dfslist["date"] = pd.DataFrame({"date":pd.date_range(dt.datetime.now().strftime('%H:%M:%S'), periods=64, freq='3h')})


In [13]:
dfslist = dfslist.resample('24h', on = 'date').mean()

In [14]:
dfslist

,Predicted Power
date,
2020-09-25,1.538387
2020-09-26,7.274228
2020-09-27,9.050759
2020-09-28,9.367571
2020-09-29,7.555704
2020-09-30,7.473544
2020-10-01,9.367571
2020-10-02,9.367571
2020-10-03,6.997674


## WindGeneration Forecast - Klushof, Germany

In [15]:
lon = ("53.56")
lat = ("8.59")

url = ("http://www.7timer.info/bin/meteo.php?lon=" +
           str(lon) + "&lat=" + str(lat)
           +"&ac=0&unit=British&output=json&tzshift=0")
    
data = requests.get(url).json()
data_size = len(data["dataseries"])

# res = [[key for key in data["dataseries"][0].keys()], *[list(idx.values()) for idx in data["dataseries"] ]]
    
# clouds = res[2][0]
    
wind =[]

    
for item in data["dataseries"]:
    wind.append(item["wind10m"])
    

wind = pd.DataFrame(wind)

In [16]:
#wind['wind speed'] = wind['speed'], replace

wind = wind.rename(columns = {'speed': 'wind speed'}, inplace = False)

In [17]:
wind

,direction,wind speed
0,220,5
1,225,5
2,230,5
3,225,5
4,210,5
...,...,...
59,230,3
60,220,3
61,200,3
62,190,3


In [18]:
fileObj = open('modelwind.obj', 'rb')
modelwind = pickle.load(fileObj)
fileObj.close()
print(modelwind)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=31, n_jobs=-1,
                      oob_score=False, random_state=None, verbose=0,
                      warm_start=False)


In [19]:
Xwind = wind[['wind speed', 'direction']].values

In [20]:
Xwind

array([[5, '220'],
       [5, '225'],
       [5, '230'],
       [5, '225'],
       [5, '210'],
       [5, '205'],
       [5, '210'],
       [5, '225'],
       [5, '220'],
       [5, '220'],
       [5, '225'],
       [5, '225'],
       [5, '205'],
       [5, '205'],
       [5, '210'],
       [5, '215'],
       [5, '220'],
       [4, '220'],
       [5, '220'],
       [4, '220'],
       [5, '200'],
       [5, '195'],
       [4, '195'],
       [4, '210'],
       [4, '210'],
       [4, '215'],
       [4, '220'],
       [4, '215'],
       [4, '205'],
       [4, '185'],
       [4, '185'],
       [4, '190'],
       [4, '195'],
       [3, '200'],
       [3, '220'],
       [3, '220'],
       [3, '200'],
       [3, '190'],
       [3, '185'],
       [4, '180'],
       [3, '175'],
       [3, '185'],
       [3, '190'],
       [3, '195'],
       [3, '190'],
       [3, '180'],
       [3, '180'],
       [3, '180'],
       [3, '185'],
       [3, '195'],
       [3, '205'],
       [3, '205'],
       [3, '

In [21]:
wind_predicted_power = modelwind.predict(Xwind)

In [22]:
wind_predicted_power

array([26.78      , 41.02935484, 40.36645161, 41.02935484, 27.21193548,
       29.15225806, 27.21193548, 41.02935484, 26.78      , 26.78      ,
       41.02935484, 41.02935484, 29.15225806, 29.15225806, 27.21193548,
       28.46612903, 26.78      , 26.78      , 26.78      , 26.78      ,
       29.15225806, 29.15225806, 29.15225806, 27.21193548, 27.21193548,
       28.46612903, 26.78      , 28.46612903, 29.15225806, 29.15225806,
       29.15225806, 29.15225806, 29.15225806, 29.15225806, 26.78      ,
       26.78      , 29.15225806, 29.15225806, 29.15225806, 29.15225806,
       29.15225806, 29.15225806, 29.15225806, 29.15225806, 29.15225806,
       29.15225806, 29.15225806, 29.15225806, 29.15225806, 29.15225806,
       29.15225806, 29.15225806, 27.21193548, 29.15225806, 29.15225806,
       29.15225806, 29.15225806, 29.15225806, 40.36645161, 40.36645161,
       26.78      , 29.15225806, 29.15225806, 29.15225806])

In [23]:
wlist = wind_predicted_power.tolist()

In [24]:
dfwlist = pd.DataFrame(wlist, columns = ['Predicted Power'])

In [25]:
dfwlist["date"] = pd.DataFrame({"date":pd.date_range(dt.datetime.now().strftime('%H:%M:%S'), periods=64, freq='3h')})

In [26]:
dfwlist = dfwlist.resample('24h', on = 'date').mean()

In [27]:
dfwlist = dfwlist.head(5)
dfwlist = dfwlist.iloc[1:]
dfwlist = dfwlist.reset_index()

In [28]:
dfwlist.columns = dfwlist.columns.str.replace(' ','_')   

In [29]:


dfwlist['day'] = pd.DatetimeIndex(dfwlist['date']).day


In [30]:
dfwlist

,date,Predicted_Power,day
0,2020-09-26,32.528024,26
1,2020-09-27,29.418992,27
2,2020-09-28,27.988347,28
3,2020-09-29,28.769960,29


In [31]:
#cols = pd.read_csv('maint.csv', nrows=0).columns.tolist()

maint = pd.read_csv('maintwind.csv')

In [32]:
maint=maint.rename(columns = {'Date Of Month':'day'})

In [33]:
maint.columns = maint.columns.str.replace(' ','_')

In [34]:
dfwlist

,date,Predicted_Power,day
0,2020-09-26,32.528024,26
1,2020-09-27,29.418992,27
2,2020-09-28,27.988347,28
3,2020-09-29,28.769960,29


In [35]:
dfwlist =dfwlist.merge(maint,left_on='day',right_on='day',how='left')

In [36]:
dfwlist

,date,Predicted_Power,day,Capacity_Available
0,2020-09-26,32.528024,26,NaN
1,2020-09-27,29.418992,27,NaN
2,2020-09-28,27.988347,28,3.0
3,2020-09-29,28.769960,29,NaN


In [37]:
dfwlist = dfwlist.fillna(10)

In [38]:
dfwlist["Predicted"] = dfwlist["Predicted_Power"] * dfwlist["Capacity_Available"] *0.1

In [39]:
dfwlist

,date,Predicted_Power,day,Capacity_Available,Predicted
0,2020-09-26,32.528024,26,10.0,32.528024
1,2020-09-27,29.418992,27,10.0,29.418992
2,2020-09-28,27.988347,28,3.0,8.396504
3,2020-09-29,28.769960,29,10.0,28.769960


In [40]:
pip install twilio

Note: you may need to restart the kernel to use updated packages.
